In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# authenticate
credential = DefaultAzureCredential()

SUBSCRIPTION="2944a580-6c5f-4258-8c86-b9c5de957998"
RESOURCE_GROUP="calebebraga08-rg"
WS_NAME="tcc-experiments"

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [2]:
# Verify that the handle works correctly.  
# If you ge an error here, modify your SUBSCRIPTION, RESOURCE_GROUP, and WS_NAME in the previous cell.
ws = ml_client.workspaces.get(WS_NAME)
print(ws.location,":", ws.resource_group)

eastus2 : calebebraga08-rg


In [3]:
TRAIN_DATAS = [
    "vrex_1999_2000_2001_2002_2003_.csv",
    "vrex_2004_2005_2006_2007_2008_.csv",
    "vrex_2009_2010_2011_2012_2013_.csv",
    "vrex_2014_2015_2016_2017_2018_.csv",
]

TEST_DATAS = [
    "vrex_2004.csv",
    "vrex_2009.csv",
    "vrex_2014.csv",
    "vrex_2019.csv"
]

version = "original"

arr_data_to_train = []
arr_data_to_test = []

for to_train, to_test in zip(TRAIN_DATAS, TEST_DATAS):
    data_to_train = ml_client.data.get(name=to_train.split('.')[0], version=version)
    arr_data_to_train.append(to_train)

    data_to_test = ml_client.data.get(name=to_test.split('.')[0], version=version)
    arr_data_to_test.append(to_test)

In [4]:
from azure.ai.ml.entities import Environment

custom_env_name = "data-prep-dependencies"

data_prep_dep = Environment(
    name=custom_env_name,
    description="Custom environment for preparation data to train and test",
    tags={"scikit-learn": "0.24.2"},
    conda_file="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/dependencies/conda_data_prep.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="initial",
)
data_prep_dep = ml_client.environments.create_or_update(data_prep_dep)

print(
    f"Environment with name {data_prep_dep.name} is registered to workspace, the environment version is {data_prep_dep.version}"
)

Environment with name data-prep-dependencies is registered to workspace, the environment version is initial


In [5]:
from azure.ai.ml.entities import Environment

custom_env_name = "train-dependencies"

train_dep = Environment(
    name=custom_env_name,
    description="Custom environment for train",
    tags={"scikit-learn": "0.24.2"},
    conda_file="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/dependencies/conda_train.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    version="initial",
)
train_dep = ml_client.environments.create_or_update(train_dep)

print(
    f"Environment with name {train_dep.name} is registered to workspace, the environment version is {train_dep.version}"
)

Environment with name train-dependencies is registered to workspace, the environment version is initial


In [6]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
data_prep_component = load_component(source="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/components/definitions/data_prep.yaml")

# Now we register the component to the workspace
data_prep_component = ml_client.create_or_update(data_prep_component)

# Create (register) the component in your workspace
print(
    f"Component {data_prep_component.name} with Version {data_prep_component.version} is registered"
)

Uploading definitions (0.0 MBs): 100%|██████████| 1321/1321 [00:00<00:00, 16021.11it/s]




Component data_prep_vrex_defaults_model with Version 1 is registered


In [7]:
# importing the Component Package
from azure.ai.ml import load_component

# Loading the component from the yml file
train_component = load_component(source="/home/azureuser/cloudfiles/code/Users/calebebraga08/src/components/definitions/train.yaml")

# Now we register the component to the workspace
train_component = ml_client.create_or_update(train_component)

# Create (register) the component in your workspace
print(
    f"Component {train_component.name} with Version {train_component.version} is registered"
)

ValidationError: Validation for ComponentSchema failed:

 {
  "result": "Failed",
  "errors": [
    {
      "message": "Environment schema data cannot be a string; In order to specify an existing environments, please provide the correct registry path prefixed with 'azureml://':\n; In order to specify an existing environments, please provide either of the following prefixed with 'azureml:':\n1. The full ARM ID for the resource, e.g.azureml:/subscriptions/<subscription_id>/resourceGroups/<resource_group>/providers/Microsoft.MachineLearningServices/workspaces/<workspace_name>/environments\n2. The short-hand name of the resource registered in the workspace, eg: azureml:<short-hand-name>:<version-if applicable>. For example, version 1 of the environment registered as 'my-env' in the workspace can be referenced as 'azureml:my-env:1'",
      "path": "environment",
      "value": "tcc-experiments:train-dependencies:initial",
      "location": "/mnt/batch/tasks/shared/LS_root/mounts/clusters/ma

In [ ]:
# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output


@dsl.pipeline(
    compute="serverless",
    description="E2E data_perp-train pipeline",
)
def prep_data_and_train_defaults_pipeline(
    data_to_train,
    data_to_test,
    learning_rate_to_train,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
    )

    train_job = train_component(
        train_data=data_prep_job.outputs.train_data,  # note: using outputs from previous step
        test_data=data_prep_job.outputs.test_data,  # note: using outputs from previous step
        learning_rate=learning_rate_to_train,  # note: using a pipeline input as parameter
    )


In [ ]:
pipelines = []

for data_to_train, data_to_test in zip(arr_data_to_train, arr_data_to_test):
    pipeline = prep_data_and_train_defaults_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        learning_rate_to_train=0.05
    )
    
    pipelines.append(pipeline)

In [ ]:
experiment_names = []
for train_name, test_name in zip(TRAIN_DATAS, TEST_DATAS):
    experiment_names.append(f"{train_name.split(".")[0]}tested_{test_name.split(".")[0]}")

for pipeline, experiment_name in zip(pipelines, experiment_names):
    pipeline_job = ml_client.create_or_update(
        pipeline,
        experiment_name=experiment_name
    )

    ml_client.jobs.stream(pipeline_job.name)